In [ ]:
from typing import Generic, TypeVar, Optional, List, Dict
from pydantic import BaseModel, parse_file_as
from google.cloud import aiplatform
from pathlib import Path
import os
class Setup(BaseModel):
    project_id: str
    region: Optional[str] 
    staging_bucket_name: str
    def init_ai(self):
        aiplatform.init(project=self.project_id, staging_bucket=self.staging_bucket_name)


# For aiplatform API guide refer https://googleapis.dev/python/aiplatform/latest/aiplatform.html
class Predict(BaseModel):
    # Expected input: Time series data. For those points that need to be predicted, leave target_column blank.
    # Note that if such points occur in the first "context_window" points used for training, errors occur.
    
    job_display_name: str # Display name of the prediction job
    model_name: str # Stores the ID of the model
    
    instances_format: str 
    # Instances for prediction, options “jsonl”, “csv”, “bigquery”, “tf-record”, “tf-record-gzip”, or “file-list”
    
    predictions_format: str 
    # Format that Vertex AI gives predictions, options “jsonl”, “csv”, or “bigquery”
    
    is_gcs_source: Optional[bool] = True 
    # If source of the instances is gcs True, if bigquery then False
    
    is_gcs_dest: Optional[bool] = True 
    # If the dump of the predictions is to be in GCS then True, else False
    
    source: str # Source URL
    
    destination_prefix: str 
    # Predictions will be dumped. Note that bucket used here must be single-region and in the same region as model.
    
    def run(self):
        attrs = vars(self).keys()
        params = {attr: getattr(self, attr) for attr in attrs if attr not in ["is_gcs_source", "is_gcs_dest", "source", "destination_prefix"]}
        if self.is_gcs_source:
            params["gcs_source"] = self.source
        else:
            params["bigquery_source"] = self.source
        if self.is_gcs_dest:
            params["gcs_destination_prefix"] = self.destination_prefix
        else:
            params["bigquery_destination_prefix"] = self.destination_prefix
        aiplatform.BatchPredictionJob.create(**params)

class Main(BaseModel):
    setup: Setup
    prediction_job: Predict
    def run(self):
        os.system("gcloud config set project " + self.setup.project_id)
        self.setup.init_ai()
        self.prediction_job.run()

filepath = "predict_automl_config.json" # Config file path for predictions
main_func = parse_file_as(Main, filepath)
main_func.run()